In [1]:
from google.colab import drive
drive.mount('/content/drive')
training_file = '/content/drive/MyDrive/all/ML/Part2/warpeace_input.txt'
raw_text = open(training_file, 'r').read()
raw_text = raw_text.lower()


Mounted at /content/drive


In [2]:
print(raw_text[:200])

the project gutenberg ebook of war and peace, by leo tolstoy

this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrict


In [3]:
all_words = raw_text.split()
unique_words = list(set(all_words))
print(f'Number of unique words: {len(unique_words)}')


Number of unique words: 45705


In [4]:
n_chars = len(raw_text)
print(f'Total characters: {n_chars}')


Total characters: 3688957


In [5]:
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print(f'Total vocabulary (unique characters): {n_vocab}')
print(chars)


Total vocabulary (unique characters): 88
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '@', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '£', '½', 'à', 'á', 'â', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'î', 'ï', 'ó', 'ô', 'ö', 'ù', 'ú', 'ü', 'ý', 'œ', '—', '‘', '’', '“', '”']


In [6]:
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_index = dict((c, i) for i, c in enumerate(chars))
print(char_to_index)


{'\n': 0, ' ': 1, '!': 2, '"': 3, '#': 4, '$': 5, '%': 6, '&': 7, "'": 8, '(': 9, ')': 10, '*': 11, ',': 12, '-': 13, '.': 14, '/': 15, '0': 16, '1': 17, '2': 18, '3': 19, '4': 20, '5': 21, '6': 22, '7': 23, '8': 24, '9': 25, ':': 26, ';': 27, '=': 28, '?': 29, '@': 30, '[': 31, ']': 32, '_': 33, 'a': 34, 'b': 35, 'c': 36, 'd': 37, 'e': 38, 'f': 39, 'g': 40, 'h': 41, 'i': 42, 'j': 43, 'k': 44, 'l': 45, 'm': 46, 'n': 47, 'o': 48, 'p': 49, 'q': 50, 'r': 51, 's': 52, 't': 53, 'u': 54, 'v': 55, 'w': 56, 'x': 57, 'y': 58, 'z': 59, '£': 60, '½': 61, 'à': 62, 'á': 63, 'â': 64, 'ä': 65, 'æ': 66, 'ç': 67, 'è': 68, 'é': 69, 'ê': 70, 'ë': 71, 'í': 72, 'î': 73, 'ï': 74, 'ó': 75, 'ô': 76, 'ö': 77, 'ù': 78, 'ú': 79, 'ü': 80, 'ý': 81, 'œ': 82, '—': 83, '‘': 84, '’': 85, '“': 86, '”': 87}


In [7]:
import numpy as np
seq_length = 160
n_seq = int(n_chars / seq_length)


In [8]:
X = np.zeros((n_seq, seq_length, n_vocab))
Y = np.zeros((n_seq, seq_length, n_vocab))


In [9]:
for i in range(n_seq):
  x_sequence = raw_text[i * seq_length : (i + 1) * seq_length]
  x_sequence_ohe = np.zeros((seq_length, n_vocab))
  for j in range(seq_length):
    char = x_sequence[j]
    index = char_to_index[char]
    x_sequence_ohe[j][index] = 1.
  X[i] = x_sequence_ohe
  y_sequence = raw_text[i * seq_length + 1 : (i + 1) * seq_length + 1] 
  y_sequence_ohe = np.zeros((seq_length, n_vocab))
  for j in range(seq_length):
    try:
      char = y_sequence[j]
    except:
      # just for ignoring out of range error
      print('final sentence')
    index = char_to_index[char]
    y_sequence_ohe[j][index] = 1.
  Y[i] = y_sequence_ohe


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses,optimizers
tf.random.set_seed(42)


In [11]:
hidden_units = 700
dropout = 0.4


In [12]:
batch_size = 100
n_epoch= 300

In [13]:
model = models.Sequential()
model.add(layers.LSTM(hidden_units, input_shape=(None, n_vocab), return_sequences=True, dropout=dropout))
model.add(layers.LSTM(hidden_units, return_sequences=True, dropout=dropout))
model.add(layers.TimeDistributed(layers.Dense(n_vocab, activation='softmax')))


In [14]:
optimizer = optimizers.RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [15]:
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, None, 700)         2209200   
                                                                 
 lstm_1 (LSTM)               (None, None, 700)         3922800   
                                                                 
 time_distributed (TimeDistr  (None, None, 88)         61688     
 ibuted)                                                         
                                                                 
Total params: 6,193,688
Trainable params: 6,193,688
Non-trainable params: 0
_________________________________________________________________
None


In [16]:
from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping


In [17]:
file_path = "/content/drive/MyDrive/all/ML/Part2/weights/weights_epoch_{epoch:03d}_loss_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='loss', verbose=1, save_best_only=True, mode='min')


In [18]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=1, mode='min')


In [19]:
def generate_text(model, gen_length, n_vocab, index_to_char):
  index = np.random.randint(n_vocab)
  y_char = [index_to_char[index]]
  X = np.zeros((1, gen_length, n_vocab))
  for i in range(gen_length):
    X[0, i, index] = 1.
    indices = np.argmax(model.predict( X[:, max(0, i - seq_length -1):i + 1, :])[0], 1)
    index = indices[-1]
    y_char.append(index_to_char[index])
  return ''.join(y_char)



In [20]:
class ResultChecker(Callback):
  def __init__(self, model, N, gen_length):
    self.model = model
    self.N = N
    self.gen_length = gen_length
    
  def on_epoch_end(self, epoch, logs={}):
    if epoch % self.N == 0:
      result = generate_text(self.model, self.gen_length, n_vocab, index_to_char)
      print('\nMy War and Peace:\n' + result)


In [21]:
result_checker = ResultChecker(model, 10, 500)

In [23]:
model.load_weights("/content/drive/MyDrive/all/ML/Part2/weights/weights_epoch_156_loss_1.6699.hdf5")
model.fit(X, Y, batch_size=batch_size, verbose=1, initial_epoch=155, epochs=n_epoch, callbacks=[result_checker, checkpoint, early_stop])


Epoch 156/300
231/231 [==============================] - ETA: 0s - loss: 1.6688
Epoch 156: loss improved from inf to 1.66876, saving model to /content/drive/MyDrive/all/ML/Part2/weights/weights_epoch_156_loss_1.6688.hdf5
231/231 [==============================] - 67s 252ms/step - loss: 1.6688
Epoch 157/300
231/231 [==============================] - ETA: 0s - loss: 1.6699
Epoch 157: loss did not improve from 1.66876
231/231 [==============================] - 63s 273ms/step - loss: 1.6699
Epoch 158/300
231/231 [==============================] - ETA: 0s - loss: 1.6675
Epoch 158: loss improved from 1.66876 to 1.66753, saving model to /content/drive/MyDrive/all/ML/Part2/weights/weights_epoch_158_loss_1.6675.hdf5
231/231 [==============================] - 62s 270ms/step - loss: 1.6675
Epoch 159/300
231/231 [==============================] - ETA: 0s - loss: 1.6659
Epoch 159: loss improved from 1.66753 to 1.66586, saving model to /content/drive/MyDrive/all/ML/Part2/weights/weights_epoch_159_lo